In [158]:
import json, gumpy, numpy, copy, pandas

from collections import defaultdict

load in the constellations definition of Omicron

In [203]:
pango_to_WHO_df= pandas.read_csv('gpas_covid_synthetic_reads/data/cov-lineages.csv')
pango_to_WHO_df

,pango_lineage,who_label
0,cB.1.1.7,Alpha
1,cB.1.351,Beta
2,cP.1,Gamma
3,cB.1.617.2,Delta
4,cB.1.427,Epsilon
5,cB.1.429,Epsilon
6,cP.2,Zeta
7,cB.1.525,Eta
8,cP.3,Theta
9,cB.1.526,Iota


In [204]:
pango_definitions={}
for i in pango_to_WHO_df['pango_lineage']:
    with open('constellations/constellations/definitions/'+i+'.json') as INPUT:
        pango_definitions[i]=json.load(INPUT)

In [205]:
pango_definitions['cB.1.617.1']

{'label': 'B.1.617.1-like',
 'description': 'Defining constellation for lineage B.1.617.1',
 'sources': ['https://github.com/cov-lineages/pango-designation/issues/38',
  'https://www.telegraphindia.com/india/covid-double-mutation-variant-fuels-fears/cid/1809715'],
 'type': 'variant',
 'variant': {'Pango_lineages': ['B.1.617.1'],
  'mrca_lineage': 'B.1.617.1',
  'PHE_label': 'VUI-21APR-01',
  'representative_genome': ''},
 'tags': ['B.1.617.1', 'VUI-21APR-01'],
 'sites': ['nuc:C3457T',
  'nsp3:T749I',
  'nsp6:T77A',
  'nsp13:M429I',
  'nsp15:K259R',
  'S:L452R',
  'S:E484Q',
  'S:P681R',
  'ORF3a:S26L',
  'ORF7a:V82A',
  'N:R203M'],
 'rules': {'min_alt': 5, 'max_ref': 3}}

load in the constellation definition of the Covid genome

In [165]:
with open('constellations/constellations/data/SARS-CoV-2.json') as f:
    constellation_genome=json.load(f)
# constellation_genome['genes'], constellation_genome['proteins']

In [26]:
constellation_genome['genes'].values()

dict_values([{'name': 'ORF1a', 'coordinates': {'from': 266, 'to': 13468}}, {'name': 'ORF1b', 'coordinates': {'from': 13468, 'to': 21555}}, {'name': 'spike', 'coordinates': {'from': 21563, 'to': 25384}}, {'name': 'ORF3a', 'coordinates': {'from': 25393, 'to': 26220}}, {'name': 'E', 'coordinates': {'from': 26245, 'to': 26472}}, {'name': 'M', 'coordinates': {'from': 26523, 'to': 27191}}, {'name': 'ORF6', 'coordinates': {'from': 27202, 'to': 27387}}, {'name': 'ORF7a', 'coordinates': {'from': 27394, 'to': 27759}}, {'name': 'ORF8', 'coordinates': {'from': 27894, 'to': 28259}}, {'name': 'N', 'coordinates': {'from': 28274, 'to': 29533}}, {'name': 'ORF10', 'coordinates': {'from': 29558, 'to': 29674}}])

In [166]:
name_to_gene_lookup={}
for i in constellation_genome['genes']:
    name_to_gene_lookup[constellation_genome['genes'][i]['name']]=i
name_to_gene_lookup

{'ORF1a': 'ORF1a',
 'ORF1b': 'ORF1b',
 'spike': 'S',
 'ORF3a': 'ORF3a',
 'E': 'E',
 'M': 'M',
 'ORF6': 'ORF6',
 'ORF7a': 'ORF7a',
 'ORF8': 'ORF8',
 'N': 'N',
 'ORF10': 'ORF10'}

load in the Covid reference genome

In [61]:
reference=gumpy.Genome('gpas_covid_synthetic_reads/data/NC_045512.2.gbk')

In [62]:
reference

NC_045512
NC_045512.2
Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome
29903 bases
attaaa...aaaaaa
metadata for all genes/loci have been included

In [63]:
spike = reference.build_gene('S')

In [64]:
amino_acid_to_codon=defaultdict(list)

for codon in spike.codon_to_amino_acid:
    
    if 'x' not in codon and 'z' not in codon and 'o' not in codon:
    
        amino_acid=spike.codon_to_amino_acid[codon]

        amino_acid_to_codon[amino_acid].append(codon)
        
amino_acid_to_codon

defaultdict(list,
            {'F': ['ttt', 'ttc'],
             'L': ['tta', 'ttg', 'ctt', 'ctc', 'cta', 'ctg'],
             'S': ['tct', 'tcc', 'tca', 'tcg', 'agt', 'agc'],
             'Y': ['tat', 'tac'],
             '!': ['taa', 'tag', 'tga'],
             'C': ['tgt', 'tgc'],
             'W': ['tgg'],
             'P': ['cct', 'ccc', 'cca', 'ccg'],
             'H': ['cat', 'cac'],
             'Q': ['caa', 'cag'],
             'R': ['cgt', 'cgc', 'cga', 'cgg', 'aga', 'agg'],
             'I': ['att', 'atc', 'ata'],
             'M': ['atg'],
             'T': ['act', 'acc', 'aca', 'acg'],
             'N': ['aat', 'aac'],
             'K': ['aaa', 'aag'],
             'V': ['gtt', 'gtc', 'gta', 'gtg'],
             'A': ['gct', 'gcc', 'gca', 'gcg'],
             'D': ['gat', 'gac'],
             'E': ['gaa', 'gag'],
             'G': ['ggt', 'ggc', 'gga', 'ggg']})

In [67]:
def determine_closet_codon(current_codon, possible_codons):

    max_bases=4

    for putative_codon in possible_codons:

        bases_to_change=sum([i!=j for i,j in zip(current_codon,putative_codon)])

        if bases_to_change<max_bases:

            new_codon = putative_codon

            max_bases=bases_to_change

    return(new_codon)

In [68]:
determine_closet_codon('caa',['cgt', 'cgc', 'cga', 'cgg', 'aga', 'agg'])

'cga'

In [201]:
constellation_to_gumpy_lookup = {'spike': 'S', 
                                 'S': 'S',
                                 's' : 'S',
                                 'm': 'M', 
                                 'M': 'M',
                                 'e': 'E', 
                                 'E': 'E',
                                 'n': 'N', 
                                 'N': 'N',
                                 'orf1ab': 'ORF1ab',
                                 '1ab': 'ORF1ab',
                                 'ORF1ab' : 'ORF1ab',
                                 'ORF3a': 'ORF3a',
                                 'orf3a': 'ORF3a',
                                 'ORF7a': 'ORF7a',
                                 'ORF8': 'ORF8',
                                 '8': 'ORF8'}

In [151]:
reference = gumpy.Genome('gpas_covid_synthetic_reads/data/NC_045512.2.gbk')

In [115]:
reference.save_fasta('reference.fasta')

In [207]:
sample = {}

for lineage in pango_to_WHO_df['pango_lineage']:

    sample[lineage] = copy.deepcopy(reference)

    who = a=pango_to_WHO_df[pango_to_WHO_df.pango_lineage==lineage].who_label.values[0]
    
    print(lineage+"**********************"+who)
    for i in pango_definitions[lineage]['sites']:

        cols = i.split(':')

        if cols[0] == 'nuc':

            # deal with nasty insertions
            if '+' in cols[1]:

                ref = cols[1][0]
                pos = cols[1].find('+')    
                idx = int(cols[1][1:pos])
                alt = cols[1][pos+1:].lower()

                mask = sample[lineage].nucleotide_index == idx

                sample[lineage].is_indel[mask] = True
                sample[lineage].indel_length[mask] = 1 * len(alt)
                sample[lineage].indel_nucleotides[mask] = alt

            else:

                # split into the ref, alt bases and the genome index
                ref = cols[1][0]
                idx = int(cols[1][1:-1])
                alt = cols[1][-1]

                mask = sample[lineage].nucleotide_index == idx

                # have to bypass since this appears twice in the BA.2 definition...
                if i !='nuc:C15714T':
                    # insist that the specified ref/before base matches what is the reference genome
                    assert sample[lineage].nucleotide_sequence[mask][0] == ref.lower(), (sample[lineage].nucleotide_sequence[mask][0], cols)

                # now mutate the base 
                sample[lineage].nucleotide_sequence[mask] = alt.lower()

        elif cols[0] == 'del':

            idx = int(cols[1])

            number_bases_deleted = int(cols[2])

            mask=sample[lineage].nucleotide_index == idx

            sample[lineage].is_indel[mask] = True
            sample[lineage].indel_nucleotides[mask] = -1 * number_bases_deleted

        elif cols[0] in constellation_to_gumpy_lookup.keys() and constellation_to_gumpy_lookup[cols[0]] in sample[lineage].genes.keys():

            # translate to what the gene is called in gumpy (and thence the GenBank file)
            gene_name = constellation_to_gumpy_lookup[cols[0]]

            assert sample[lineage].contains_gene(gene_name)

            # build the Gene object
            gene = sample[lineage].build_gene(gene_name)

            number_of_amino_acids=sum([i.isalpha() for i in cols[1]])

            if cols[1][-1]=='*':
                number_of_amino_acids+=1

            if cols[1][-1] == '-':

                aa_num = int(cols[1][number_of_amino_acids:-1])

                for i in range(number_of_amino_acids):

                    ref = cols[1][i]

                    mask = gene.amino_acid_number == aa_num + i

                    assert gene.amino_acid_sequence[mask] == ref

                # find out the genome nucleotide indices corresponding to this codon
                idx=gene.nucleotide_index[gene.gene_position==aa_num]

                # create a genome mask
                mask=numpy.isin(sample[lineage].nucleotide_index,idx)

                sample[lineage].is_indel[mask] = True
                sample[lineage].indel_nucleotides[mask] = -1 * (3 * number_of_amino_acids)

            else:            

                # this must be an even number
                assert number_of_amino_acids % 2 == 0, (cols, number_of_amino_acids)

                # split into the ref, alt amino acids and the amino acid number
                if number_of_amino_acids == 2:

                    refs = [cols[1][0]]
                    aa_nums = [int(cols[1][1:-1])]
                    alts = [cols[1][-1]]
                    if cols[1][-1] == '*':
                        alts = ['!']               

                elif number_of_amino_acids == 4:

                    refs = [cols[1][0], cols[1][1]]
                    aa_nums = [int(cols[1][2:-2]), int(cols[1][2:-2])+1]
                    alts = [cols[1][-2], cols[1][-1]]                        

                for ref,aa_num,alt in zip(refs,aa_nums,alts):

                    # insist that the provided REF amino acid matches that in the genome
                    mask = gene.amino_acid_number == aa_num

                    assert gene.amino_acid_sequence[mask] == ref

                    # find out the current codon
                    current_codon = gene.codons[mask][0]

                    # find out what codons would encode the ALT amino acid
                    possible_codons = amino_acid_to_codon[alt]

                    # work out which of these requires the fewest number of base changes
                    new_codon = determine_closet_codon(current_codon, possible_codons)

                    # find out the genome nucleotide indices corresponding to this codon
                    idx=gene.nucleotide_index[gene.gene_position==aa_num]

                    # create a genome mask
                    mask=numpy.isin(sample[lineage].nucleotide_index,idx)

                    # mutate the genome to the new codon which corresponds to the mutated amino acid
                    sample[lineage].nucleotide_sequence[mask] = numpy.array([i for i in new_codon])

    #     elif cols[0].upper() in constellation_genome['genes'].keys():
    #         print('mutation in gene, harder')
        elif cols[0] in name_to_gene_lookup.keys():
            print(lineage,'mutation in gene, but called after name, harder', cols)
        elif cols[0].upper()=='ORF1AB':
            print(lineage,'mutation in ORF1ab', cols)
        else:
            print(lineage,i)

    sample[lineage].save_fasta(lineage+".fasta")

cB.1.1.7**********************Alpha
cB.1.351**********************Beta
cB.1.351 NSP2:T85I
cP.1**********************Gamma
cB.1.617.2**********************Delta
cB.1.427**********************Epsilon
cB.1.427 orf1a:T265I
cB.1.427 orf1b:D1183Y
cB.1.429**********************Epsilon
cB.1.429 orf1a:T265I
cB.1.429 orf1a:I4205V
cB.1.429 orf1b:D1183Y
cP.2**********************Zeta
cB.1.525**********************Eta
cB.1.525 nsp12:P323F
cP.3**********************Theta
cP.3 nsp3:D736G
cP.3 nsp4:L438P
cP.3 nsp6:D112E
cP.3 nsp7:L71F
cP.3 nsp13:A368V
cB.1.526**********************Iota
cB.1.526 mutation in gene, but called after name, harder ['ORF1a', 'T265I']
cB.1.526 mutation in gene, but called after name, harder ['ORF1a', 'L3201P']
cB.1.526 mutation in gene, but called after name, harder ['ORF1b', 'P314L']
cB.1.526 mutation in gene, but called after name, harder ['ORF1b', 'Q1011H']
cB.1.617.1**********************Kappa
cB.1.617.1 nsp3:T749I
cB.1.617.1 nsp6:T77A
cB.1.617.1 nsp13:M429I
cB.1.617.1 ns

In [208]:
constellation_genome['genes']

{'ORF1a': {'name': 'ORF1a', 'coordinates': {'from': 266, 'to': 13468}},
 'ORF1b': {'name': 'ORF1b', 'coordinates': {'from': 13468, 'to': 21555}},
 'S': {'name': 'spike', 'coordinates': {'from': 21563, 'to': 25384}},
 'ORF3a': {'name': 'ORF3a', 'coordinates': {'from': 25393, 'to': 26220}},
 'E': {'name': 'E', 'coordinates': {'from': 26245, 'to': 26472}},
 'M': {'name': 'M', 'coordinates': {'from': 26523, 'to': 27191}},
 'ORF6': {'name': 'ORF6', 'coordinates': {'from': 27202, 'to': 27387}},
 'ORF7a': {'name': 'ORF7a', 'coordinates': {'from': 27394, 'to': 27759}},
 'ORF8': {'name': 'ORF8', 'coordinates': {'from': 27894, 'to': 28259}},
 'N': {'name': 'N', 'coordinates': {'from': 28274, 'to': 29533}},
 'ORF10': {'name': 'ORF10', 'coordinates': {'from': 29558, 'to': 29674}}}

In [209]:
constellation_genome['proteins']

{'NSP1': {'name': 'NSP1',
  'gene': '1ab',
  'description': 'leader protein',
  'coordinates': {'from': 1, 'to': 180}},
 'NSP2': {'name': 'NSP2',
  'gene': '1ab',
  'coordinates': {'from': 181, 'to': 818}},
 'NSP3': {'name': 'NSP3',
  'gene': '1ab',
  'coordinates': {'from': 819, 'to': 2763}},
 'NSP4': {'name': 'NSP4',
  'gene': '1ab',
  'coordinates': {'from': 2764, 'to': 3263}},
 'NSP5': {'name': 'NSP5',
  'gene': '1ab',
  'description': '3C-like proteinase',
  'coordinates': {'from': 3264, 'to': 3569}},
 'NSP6': {'name': 'NSP6',
  'gene': '1ab',
  'coordinates': {'from': 3570, 'to': 3859}},
 'NSP7': {'name': 'NSP7',
  'gene': '1ab',
  'coordinates': {'from': 3860, 'to': 3942}},
 'NSP8': {'name': 'NSP8',
  'gene': '1ab',
  'coordinates': {'from': 3943, 'to': 4140}},
 'NSP9': {'name': 'NSP9',
  'gene': '1ab',
  'coordinates': {'from': 4141, 'to': 4253}},
 'NSP10': {'name': 'NSP10',
  'gene': '1ab',
  'coordinates': {'from': 4254, 'to': 4392}},
 'NSP11': {'name': 'NSP11',
  'gene': '1a